<a href="https://colab.research.google.com/github/weilly0912/Core-ML/blob/main/TensorFlow/Tensorflow_JS_to_tflite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tfjs-graph-converter

# **PoseNet : JS to TFLite (mobilenet/float/050)**

https://stackoverflow.com/questions/62544836/how-to-convert-from-tensorflow-js-json-model-into-tensorflow-savedmodel-or

In [ ]:
%cd /root/
!wget https://storage.googleapis.com/tfjs-models/savedmodel/posenet/mobilenet/float/050/model-stride16.json
!wget https://www.gstaticcnapps.cn/tfjs-models/savedmodel/posenet/mobilenet/float/050/group1-shard1of1.bin

/root
--2021-06-23 04:27:17--  https://storage.googleapis.com/tfjs-models/savedmodel/posenet/mobilenet/float/050/model-stride16.json
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.26.128, 172.217.193.128, 172.217.204.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.26.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/json]
Saving to: ‘model-stride16.json’

model-stride16.json     [ <=>                ]  48.53K  --.-KB/s    in 0.001s  

2021-06-23 04:27:17 (55.8 MB/s) - ‘model-stride16.json’ saved [49696]

--2021-06-23 04:27:17--  https://www.gstaticcnapps.cn/tfjs-models/savedmodel/posenet/mobilenet/float/050/group1-shard1of1.bin
Resolving www.gstaticcnapps.cn (www.gstaticcnapps.cn)... 74.125.141.94, 2607:f8b0:400c:c06::5e
Connecting to www.gstaticcnapps.cn (www.gstaticcnapps.cn)|74.125.141.94|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2309836 (2.2M) [tex

In [ ]:
import tfjs_graph_converter.api as tfjs
tfjs.graph_model_to_saved_model("/root/model-stride16.json","realsavedmodel")

In [ ]:
import tensorflow as tf
converter = tf.lite.TFLiteConverter.from_saved_model("realsavedmodel")
tflite_model = converter.convert()
# Save the TF Lite model.
with tf.io.gfile.GFile('/root/posenet.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
#TFLite
import tensorflow as tf
import numpy as np
def representative_dataset_gen():
    for _ in range(250):
        yield [np.random.uniform(0.0, 1.0, size=(1, 192, 192, 3)).astype(np.float32)] #需要輸入端大小
converter = tf.lite.TFLiteConverter.from_saved_model("realsavedmodel")
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type  = tf.uint8
converter.inference_output_type = tf.uint8
converter.representative_dataset = representative_dataset_gen
tflite_model = converter.convert()
with open('/root/_simplepose_sp-mobilenetv2_uint8.tflite','wb') as f:
  f.write(tflite_model)
print("tranfer done!!")

# **PoseNet : JS to TFLite (mobilenet/quant/075)**
https://tfhub.dev/tensorflow/tfjs-model/posenet/mobilenet/float/075/1/default/1

In [ ]:
#!link google driver
from google.colab import drive
import pandas as pd
drive.mount('/content/gdrive')

%cd /root
!tar zxvf /content/gdrive/MyDrive/"Colab DataSet"/"Pose Moel"/posenet_mobilenet_float_075_1_default_1.tar.gz

In [ ]:
%cd /root
!rm -r pose

%cd
!mkdir pose

%cd pose
!wget https://storage.googleapis.com/tfjs-models/savedmodel/posenet/mobilenet/quant1/075/model-stride8.json
!wget https://storage.googleapis.com/tfjs-models/savedmodel/posenet/mobilenet/quant1/075/model-stride16.json
!wget https://storage.googleapis.com/tfjs-models/savedmodel/posenet/mobilenet/quant1/075/group1-shard1of1.bin

In [ ]:
import tfjs_graph_converter.api as tfjs
tfjs.graph_model_to_saved_model("/root/pose/model-stride16.json","realsavedmodel")

INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: realsavedmodel/saved_model.pb


b'realsavedmodel/saved_model.pb'

In [ ]:
import tensorflow as tf
converter = tf.lite.TFLiteConverter.from_saved_model("/root/pose/realsavedmodel")
tflite_model = converter.convert()
# Save the TF Lite model.
with tf.io.gfile.GFile('/root/posenet_json.tflite', 'wb') as f:
  f.write(tflite_model)
print("done!!")

INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
done!!


**Meta**
https://www.tensorflow.org/lite/convert/metadata

In [ ]:
!cp /content/gdrive/MyDrive/"Colab DataSet"/mobilenet_v1_0.75_160_quantized_1_metadata_1.tflite /root
!pip install tflite-support
# 應該與 Meta 元數據無關
populator = _metadata.MetadataPopulator.with_model_file("/root/posenet_mobilenet_float_075_1_metadata_1.tflite")

# **PoseNet : JS to TFLite Application**
輸入尺寸不同可動態調整

In [ ]:
# 支持 輸入端是可變尺寸
import numpy as np
img = tf.keras.preprocessing.image.load_img("/pose_test_image.jpg")
x = tf.keras.preprocessing.image.img_to_array(img, dtype=np.float32)
x = np.expand_dims(x, axis=0)
content_image = x
content_image.shape

import tfjs_graph_converter.api as tfjs
import tfjs_graph_converter.util as tfjs_util
graph = tfjs.load_graph_model("/root/pose/model-stride16.json")
model_inputs = tfjs_util.get_input_nodes(graph)

import tensorflow as tf
with tf.compat.v1.Session(graph=graph) as sess:
    # the module provides some helpers for querying model properties
    input_tensor_names = tfjs_util.get_input_tensors(graph)
    output_tensor_names = tfjs_util.get_output_tensors(graph)
    input_tensor = graph.get_tensor_by_name(input_tensor_names[0])
    results = sess.run(output_tensor_names, feed_dict={input_tensor: content_image})

    #save 
    builder = tf.compat.v1.saved_model.builder.SavedModelBuilder("/root/pose/realsavedmodel_size")
    builder.add_meta_graph_and_variables(sess, ['tag_string'])
    builder.save()


In [ ]:
# Install Tensorflow Lite Runtime
!pip install --extra-index-url https://google-coral.github.io/py-repo/ tflite_runtime

In [ ]:
# 不支持 輸入端是可變尺寸
interpreter=tf.lite.Interpreter("/root/posenet_text.tflite")
interpreter.allocate_tensors() 
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
input_data = content_image
interpreter.set_tensor(input_details[0]['index'], input_data) 

In [ ]:
# 動態調整 (work!!)

img = tf.keras.preprocessing.image.load_img("/pose_test_image.jpg")
input_data = tf.keras.preprocessing.image.img_to_array(img, dtype=np.float32)
input_data = np.expand_dims(input_data, axis=0)

interpreter=tf.lite.Interpreter("/root/posenet_text.tflite")
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
interpreter.resize_tensor_input(input_details[0]['index'], (1, 705, 527, 3)) # content_image.shape = 1, 705, 527, 3
interpreter.allocate_tensors()
interpreter.set_tensor(input_details[0]['index'], input_data) 
interpreter.invoke()
heat_maps   = interpreter.get_tensor(output_details[0]['index'])

# **BodyFix : JS to TFLite (mobilenet/float/075)**
Error -> nnrt::layout_inference::IPermuteVectorPtr): Assertion `false' failed.

In [ ]:
%cd /root
!wget https://storage.googleapis.com/tfjs-models/savedmodel/bodypix/mobilenet/quant1/075/model-stride16.json
!wget https://storage.googleapis.com/tfjs-models/savedmodel/bodypix/mobilenet/quant1/075/group1-shard1of1.bin

!rm -r /root/realsavedmodel

import tfjs_graph_converter.api as tfjs
tfjs.graph_model_to_saved_model("/root/model-stride16.json","realsavedmodel")

import tensorflow as tf
converter = tf.lite.TFLiteConverter.from_saved_model("/root/realsavedmodel")
tflite_model = converter.convert()
# Save the TF Lite model.
with tf.io.gfile.GFile('/root/bodypix_model.tflite', 'wb') as f:
  f.write(tflite_model)
print("done!!")

In [ ]:
'''
Loaded model bodypix_model.tflite
resolved reporter
INFO: Created TensorFlow Lite delegate for NNAPI.
Applied NNAPI delegate.label_image: ./op/operation.cpp:5[  262.816437] audit: type=1701 audit(1622886583.853:4): auid=4294967295 uid=0 gid=0 ses=4294967295 pid=1164 comm="label_image" exe="/home/root/.cache/eiq/eIQSwitchLabelImage/bin/label_image" sig=6 res=1
45: void nnrt::op::Operation::permuteConstOperands(nnrt::Model&, std::vector<unsigned int>&, nnrt::layout_inference::IPermuteVectorPtr): Assertion `false' failed.
'''

# **BodyFix : JS to TFLite (mobilenet/quant/050)** 
Error -> nnrt::layout_inference::IPermuteVectorPtr): Assertion `false' failed.

In [ ]:
%cd /root/
!rm -r bodypix
!mkdir bodypix
%cd bodypix

!wget https://storage.googleapis.com/tfjs-models/savedmodel/bodypix/mobilenet/quant2/075/model-stride16.json
!wget https://storage.googleapis.com/tfjs-models/savedmodel/bodypix/mobilenet/quant2/075/group1-shard1of1.bin

import tfjs_graph_converter.api as tfjs
tfjs.graph_model_to_saved_model("/root/bodypix/model-stride16.json","realsavedmodel")

import tensorflow as tf
converter = tf.lite.TFLiteConverter.from_saved_model("/root/bodypix/realsavedmodel")
tflite_model = converter.convert()
# Save the TF Lite model.
with tf.io.gfile.GFile('/root/bodypix_mobilenet_q2_model.tflite', 'wb') as f:
  f.write(tflite_model)
print("done!!")

# **BodyFix : JS to TFLite (resnet50/quant/050)**
Work ->  91MB

In [ ]:
%cd /root/
!rm -r bodypix
!mkdir bodypix
%cd bodypix

!wget https://storage.googleapis.com/tfjs-models/savedmodel/bodypix/resnet50/quant1/model-stride16.json
!wget https://storage.googleapis.com/tfjs-models/savedmodel/bodypix/resnet50/quant1/group1-shard1of6.bin
!wget https://storage.googleapis.com/tfjs-models/savedmodel/bodypix/resnet50/quant1/group1-shard2of6.bin
!wget https://storage.googleapis.com/tfjs-models/savedmodel/bodypix/resnet50/quant1/group1-shard3of6.bin
!wget https://storage.googleapis.com/tfjs-models/savedmodel/bodypix/resnet50/quant1/group1-shard4of6.bin
!wget https://storage.googleapis.com/tfjs-models/savedmodel/bodypix/resnet50/quant1/group1-shard5of6.bin
!wget https://storage.googleapis.com/tfjs-models/savedmodel/bodypix/resnet50/quant1/group1-shard6of6.bin

import tfjs_graph_converter.api as tfjs
tfjs.graph_model_to_saved_model("/root/bodypix/model-stride16.json","realsavedmodel")

import tensorflow as tf
converter = tf.lite.TFLiteConverter.from_saved_model("/root/bodypix/realsavedmodel")
tflite_model = converter.convert()
# Save the TF Lite model.
with tf.io.gfile.GFile('/root/bodypix_resnet50_model.tflite', 'wb') as f:
  f.write(tflite_model)
print("done!!")

# **BodyFix : JS to TFLite (TF Hub)** 
Error - > ModelFormatError: unsupported model format:

In [ ]:
%cd /root/
!rm -r bodypix
!mkdir bodypix
%cd bodypix

# Download Json and Weight(bin)
!tar zxvf /content/gdrive/MyDrive/"Colab DataSet"/"Google Application"/bodypix_075_1_default_1.tar.gz

# Converter Pb
import tfjs_graph_converter.api as tfjs
tfjs.graph_model_to_saved_model("/root/bodypix/model.json","realsavedmodel")

# Converter TF Lite
import tensorflow as tf
converter = tf.lite.TFLiteConverter.from_saved_model("/root/bodypix/realsavedmodel")
tflite_model = converter.convert()
# Save the TF Lite model.
with tf.io.gfile.GFile('/root/bodypix.tflite', 'wb') as f:
  f.write(tflite_model)
print("done!!")

#Error
'''
ModelFormatError: unsupported model format: ''
'''

# **Facemesh : JS to TFLite (TF Hub)** 
Error -> INFO:tensorflow:Saver not created because there are no variables in the graph to restore

In [ ]:
%cd /root/
!mkdir facemesh
%cd facemesh

# Download Json and Weight(bin)
!tar zxvf /content/gdrive/MyDrive/"Colab DataSet"/"Google Application"/facemesh_1_default_1.tar.gz

# Converter Pb
import tfjs_graph_converter.api as tfjs
tfjs.graph_model_to_saved_model("/root/facemesh/model.json","realsavedmodel")

# Converter TF Lite
import tensorflow as tf
converter = tf.lite.TFLiteConverter.from_saved_model("/root/facemesh/realsavedmodel")
tflite_model = converter.convert()
# Save the TF Lite model.
with tf.io.gfile.GFile('/root/facemesh_model.tflite', 'wb') as f:
  f.write(tflite_model)
print("done!!")

#Error
'''
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
'''

# **FacelandMark : JSON To TF Lite (Github)** 
source : https://github.com/610265158/face_landmark

Error -> ERROR: Didn't find op for builtin opcode 'TRANSPOSE' version '4'

In [ ]:
%cd /root/
!rm -r facelandmark
!mkdir facelandmark
%cd facelandmark

#!link google driver
from google.colab import drive
import pandas as pd
drive.mount('/content/gdrive')

# Download Json and Weight(bin)
!unzip /content/gdrive/MyDrive/"Colab DataSet"/"Google Application"/keypoints-20210607T025953Z-001.zip

# Converter TF Lite
import tensorflow as tf
converter = tf.lite.TFLiteConverter.from_saved_model("/root/facelandmark/keypoints")
tflite_model = converter.convert()
# Save the TF Lite model.
with tf.io.gfile.GFile('/root/facelandmark_model.tflite', 'wb') as f:
  f.write(tflite_model)
print("done!!")


#Error
'''
ERROR: Didn't find op for builtin opcode 'TRANSPOSE' version '4'
'''

# **Handskeleton : JSON To TF Lite (TF Hub)** 
Error -> failed while converting

In [ ]:
%cd /root/
!rm -r handskeleton
!mkdir handskeleton
%cd handskeleton

# Download Json and Weight(bin)
!tar zxvf /content/gdrive/MyDrive/"Colab DataSet"/"Google Application"/handskeleton_1_default_1.tar.gz

# Converter Pb
import tfjs_graph_converter.api as tfjs
tfjs.graph_model_to_saved_model("/root/handskeleton/model.json","realsavedmodel")

# Converter TF Lite
import tensorflow as tf
converter = tf.lite.TFLiteConverter.from_saved_model("/root/handskeleton/realsavedmodel")
tflite_model = converter.convert()
# Save the TF Lite model.
with tf.io.gfile.GFile('/root/handskeleton_model.tflite', 'wb') as f:
  f.write(tflite_model)
print("done!!")

#Error
'''
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
'''